In [38]:
pred_treatment_effect_file1 = "/data2/wuyinjun/causal_tabular/ihdp/seed_0/ours/logs5/3/pred_treatment_effect.json"
rule_pred_treatment_effect_file1 = "/data2/wuyinjun/causal_tabular/ihdp/seed_0/ours/logs5/3/rule_pred_treatment_effect.json"
pred_treatment_effect_file2 = "/data2/wuyinjun/causal_tabular/ihdp/seed_0/TransTEE/logs/3/pred_treatment_effect.json"
gt_treatment_effect_f = "/data2/wuyinjun/causal_tabular/ihdp/seed_0/ours/logs5/3/gt_treatment_effect.json"


import json
import numpy as np
with open(pred_treatment_effect_file1, 'r') as f:
    pred_treatment_effect1 = json.load(f)

with open(rule_pred_treatment_effect_file1, 'r') as f:
    rule_pred_treatment_effect1 = json.load(f)

with open(pred_treatment_effect_file2, 'r') as f:
    pred_treatment_effect2 = json.load(f)
with open(gt_treatment_effect_f, 'r') as f:
    gt_treatment_effect = json.load(f)

better_perf_count = 0
largest_diff = 0
largest_diff_key = None
for key in pred_treatment_effect1:
    pred_te1 = pred_treatment_effect1[key]
    if type(pred_te1) == list:
        pred_te1 = pred_te1[0]
    pred_te2 = pred_treatment_effect2[key]
    if type(pred_te2) == list:
        pred_te2 = pred_te2[0]
    gt_te = gt_treatment_effect[key]
    if type(gt_te) == list:
        gt_te = gt_te[0]

    diff1 = np.abs(pred_te1 - gt_te)
    diff2 = np.abs(pred_te2 - gt_te)
    if diff1 < diff2:
        print(key, diff1, diff2, gt_te, pred_te1, pred_te2)
        better_perf_count += 1
        if diff2 - diff1 > largest_diff:
            largest_diff = diff2 - diff1
            largest_diff_key = key

print(better_perf_count, len(pred_treatment_effect1))
print(largest_diff_key, largest_diff)


0 2.3752546310424805 2.4415059089660645 2.4810099601745605 4.856264591217041 4.922515869140625
1 0.6585054397583008 0.6912083625793457 5.290266036987305 4.631760597229004 4.599057674407959
4 0.14794301986694336 0.22555303573608398 4.33616304397583 4.484106063842773 4.561716079711914
5 2.003192186355591 2.0170557498931885 3.081373929977417 5.084566116333008 5.0984296798706055
6 1.8297603130340576 1.8761231899261475 5.542995452880859 3.7132351398468018 3.666872262954712
7 0.5958347320556641 0.6285066604614258 4.187276363372803 3.5914416313171387 3.558769702911377
9 1.7790460586547852 1.8806486129760742 5.8562822341918945 4.077236175537109 3.9756336212158203
11 0.9215240478515625 1.0023689270019531 4.635707855224609 3.714183807373047 3.6333389282226562
12 0.3045539855957031 0.3769559860229492 4.443111419677734 4.7476654052734375 4.820067405700684
13 0.4788827896118164 0.6598005294799805 3.623288154602051 3.1444053649902344 2.9634876251220703
14 1.892162799835205 1.9177565574645996 6.14210

In [43]:
explanation_f = "/data2/wuyinjun/causal_tabular/ihdp/seed_0/ours/logs5/3/explanation.json"
with open(explanation_f, 'r') as f:
    explanation_mappings = json.load(f)

target_key = 33
for exp in explanation_mappings:
    sample_idx_ls = explanation_mappings[exp]

    if int(target_key) in sample_idx_ls:
        print(exp, sample_idx_ls)
        all_pred_te1 = 0
        all_pred_te2 = 0
        all_gt_te = 0
        for idx in sample_idx_ls:
            idx = str(idx)
            print(idx, pred_treatment_effect1[idx], rule_pred_treatment_effect1[idx], pred_treatment_effect2[idx], gt_treatment_effect[idx])
            print((pred_treatment_effect2[idx]*8 + rule_pred_treatment_effect1[idx][0])/(1 + 8))
            if type(pred_treatment_effect1[idx]) == list:
                all_pred_te1 += pred_treatment_effect1[idx][0]
            else:
                all_pred_te1 += pred_treatment_effect1[idx]
            if type(pred_treatment_effect2[idx]) == list:
                all_pred_te2 += pred_treatment_effect2[idx][0]
            else:
                all_pred_te2 += pred_treatment_effect2[idx]
            
            if type(gt_treatment_effect[idx]) == list:
                all_gt_te += gt_treatment_effect[idx][0]
            else:
                all_gt_te += gt_treatment_effect[idx]
            # all_pred_te1 += pred_treatment_effect1[idx]
            # all_pred_te2 += pred_treatment_effect2[idx]
            # all_gt_te += gt_treatment_effect[idx]
        print(all_pred_te1/len(sample_idx_ls), all_pred_te2/len(sample_idx_ls), all_gt_te/len(sample_idx_ls))
        print(np.abs(all_pred_te1-all_gt_te), np.abs(all_pred_te2 - all_gt_te))
        break

ark>=0.5959596037864685 cig<=0.1111111119389534 drugs>=0.5959596037864685 momage<=1.0108255581773689 momage>=-0.9297208373009376 tex<=0.5959596037864685 [29, 33, 101]
29 [5.123590469360352] [3.333404302597046] 5.3266754150390625 4.2440972328186035
5.105200846989949
33 [4.104948997497559] [3.333404302597046] 4.165709495544434 3.209796905517578
4.073231140772502
101 [3.265287399291992] [3.333404302597046] 3.17116641998291 3.3850579261779785
3.189192851384481
4.164608955383301 4.221183776855469 3.61298402150472
1.6548748016357422 1.824599266052246


In [33]:
print(all_pred_te1*(1 + 8) - all_pred_te2*8)
key = 101
print(pred_treatment_effect1[str(key)][0]*(1 + 8) - pred_treatment_effect2[str(key)]*8)

11.136031150817871
4.018255233764648
